# Audio analysis with Librosa

CS224S / LINGUIST285: Spoken Language Processing. Homework 1 Notebook.
 

### Instructions

Throughout the notebook, there will be blocks labeled with **HW Task**, which have cells left open for your solutions. The tasks in this notebook make up 50 points of the overall 100 for Homework 1.

### Test your installation

Ensure you can run all of the imports below before you start work. Run these imports as part of your initial session each time. In this notebook we will use the library [Librosa](https://librosa.org/doc/latest/index.html) which has some specialized functions for working with audio. You should have already set up the library in your python environment by following the homework handout instructions.

In [ ]:
import librosa
import librosa.display
import librosa.effects
import librosa.util

# also check you can import numpy and matplotlib, which are part of the anaconda package
import numpy as np
import matplotlib.pyplot as plt

---

## A new speech dataset

Throughout this course, we will be working with a new speech dataset of conversations between bank employees and customers. In this homework, we will take our first glance at a handful of raw audio clips from the [HarperValleyBank dataset](https://arxiv.org/abs/2010.13929). Working with audio files in this way is similar to what you might experience when working with data exported from call center recordings or similar telephone/app-based human-human interactions. 

To start, ensure you can execute the commands below to locate and load the audio files.

In [ ]:
ls sample

In the `sample` folder, we've put a few example conversations between an agent (bank employee) and a caller (bank customer). Each conversation is split into two `.wav` files by speaker. For example, `sample/agent/0002f70f7386445b.wav` and `sample/caller/0002f70f7386445b.wav` belong to the same conversation. The first file contains everything the agent says while the second contains everything the caller says. This speaker-separated format is common in speech corpora.

In [ ]:
ls sample/agent

In [ ]:
ls sample/caller

We can listen to the audio files to familiarize ourselves with the data. Ensure your Jupyter setup allows this as you will need to listen to audio to complete the assignment. The long pauses you hear between utterances is because each audio file is from only one side of the conversation.

In [ ]:
import IPython.display as ipd

In [ ]:
# agent part of the conversation
ipd.Audio('./sample/agent/0002f70f7386445b.wav')

In [ ]:
# caller part of the conversation
ipd.Audio('./sample/caller/0002f70f7386445b.wav')

When working with spoken language, we often want to visualize and process the raw audio before applying machine learning or signal processing approaches. 

The `librosa` package contains a suite of utilities to open and process waveforms. The function `librosa.load` loads an audio file as a floating point time series. It will return a numpy file containing samples from the audio clip and a sample rate (`sr`) used to discretize the continuous waveform. 

In [ ]:
# sr represents a "sample rate" that converts continuous audio data to a tensor. 
# the higher the sample rate, the "higher resolution" the audio clip is.
wav_agent, sr_agent = librosa.load('./sample/agent/0002f70f7386445b.wav')
wav_caller, sr_caller = librosa.load('./sample/caller/0002f70f7386445b.wav')

Like above, there are long pauses in each clip where the other speaker is speaking.  These "silence" gaps are intended to allow users to recover the original conversation if needed.

**HW Task (2 points):** Your first task (to familiarize yourself with waveforms and `librosa`) is to merge the agent and caller waveforms into a single waveform. Use `ipd.Audio` to play the merged waveform in this notebook. You may assume that the audio files to combine will always start with the same sampling rate `(sr_caller == sr_agent)`

In [ ]:
##
#
# YOUR CODE HERE
#
##

wav_total = None # TODO: define this variable
sr_total = sr_agent

In [ ]:
# helper function to plot a mel spectrogram
# arguments: (wave array, sampling rate, number of mel bins, max frequency of mel scale)
def plot_melspectrogram(wav, sr, annotations=None, n_mels=256, fmax=4096):
    
    fig, ax = plt.subplots(1,1,figsize=(20,5))
    M = librosa.feature.melspectrogram(y=wav, sr=sr, n_mels=n_mels, fmax=fmax, n_fft=2048)
    M_db = librosa.power_to_db(M, ref=np.max)
    img = librosa.display.specshow(M_db, y_axis='mel', x_axis='time', ax=ax, fmax=fmax)
    ax.set(title='Mel spectrogram display')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    
    # iterate over list of text annotations and draw them
    if annotations is not None:
        for x,y,text in annotations:
            ax.annotate(
            text,
            xy=(x,y), xycoords='data',
            xytext=(10, -50), textcoords='offset pixels',
            horizontalalignment='right',
            color='white',
            fontsize=20,
            verticalalignment='bottom',
            arrowprops=dict(
                arrowstyle= '-|>',
                 color='white',
                 lw=1,
                 ls='-')
            ) 

Let's use this helper function to visualize your merged audio file. Note the keywords used below. First we will set the max frequency considered for our mel scale (`fmax`) to the max possible frequency for this sampling rate (Nyquist Frequency, `sample_rate/2`). The number of bins is effectively the "resolution" in mel scale of the vertical axis.

In [ ]:
plot_melspectrogram(wav_total, sr_agent, annotations=None, n_mels=256, fmax=sr_agent/2)

Notice there is no energy above 4kHz. The dataset was collected using telephone speech, which has a sampling rate of 8kHz. Therefore the highest meaningful frequency in these recordings is 4kHz. Use the plotting arguments below as your default when displaying audio to hand in. For convenience we also set these as the default for the helper function.

In [ ]:
# we can adjust the keyword arguments
fmax_dataset = 4096
plot_melspectrogram(wav_total, sr_agent, annotations=None, n_mels=256, fmax=fmax_dataset)
plot_melspectrogram(wav_total, sr_agent)

**HW Task (3 points)**: Often, we may wish to focus on a single speaker, rather than the conversation. Focusing on the **agent**, we want to remove the silences from the raw .wav file. This will collapse all of the agent's utterances into a single array with just short pauses between utterances. Removing silences in this way helps focus on the speech parts of an audio file. Sometimes this process is called _voice activity detection_, especially in scenarios with strong background noise or distortion when speech can be hard to identify from the background.  

To do this:

1. Use `librosa.effects.split` to split the agent .wav file by silence. 
2. Clip out the silences and combine the audio back into a single array with silences removed. It should sound like a more or less continuous utterance stream from the speaker. Ensure you aren't filtering too aggressively and clipping out actual audio. 

(you are of course free to try other approaches to silence removal. Describe your approach briefly along with the implementation)

In [ ]:
##
#
# YOUR CODE HERE
#
##

recon_agent = None # TODO: represent as a NumPy array

In [ ]:
# listen to your audio file
ipd.Audio(recon_agent, rate=sr_agent)

In [ ]:
# This should now have minimal time gaps between utterances
plot_melspectrogram(recon_agent, sr_agent)

Finally, an entire audio signal is too long, so let's pick a few utterances for further work.

**HW Task (5 points)**: In the conversation above, the agent says the following: `This is Harper Valley National Bank. My name is Elizabeth. How can I help you today?`. Create a single wave array for just this part of the utterance and visualize it.

In [ ]:
##
#
# YOUR CODE HERE
#
##

audio_signal = None # TODO: represent as a NumPy array

In [ ]:
# listen to your audio file
ipd.Audio(audio_signal, rate=sr_agent)

In [ ]:
plot_melspectrogram(audio_signal, sr_agent)

**HW Task (5 points)**: Now use what you know about how words and phones appear in time-frequency space to label the words in this utterance. Below is a first annotation to demonstrate the format

In [ ]:
##
#
# YOUR CODE HERE
#
##

# annotation tuples assume the form (time (s), frequency(Hz), annotation(string))
utt_words = [(0.1, 1024, 'This'),]

In [ ]:
plot_melspectrogram(audio_signal, sr_agent, annotations=utt_words)

**HW Task (5 points)**: Estimate the mean F_0 frequency during the word national. Does the F_0 curve have high variance during this word? (it's okay to estimate the mean F_0 segment visually)

In [ ]:
##
#
# YOUR CODE HERE
#
##

We will use the snippet `audio_signal` in all the exercises below.

---

We've been visualizing our utterances in Mel Spectrograms because it is easier to "read" speech-relevant phones in this view. Now let's look at the raw waveform time series plot itself

In [ ]:
# matplotlib will have been installed upon conda environment creation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

def show_waveplot(audio_signal,sr,annotations=None):
    fig, ax = plt.subplots(1,1,figsize=(20,5))
    librosa.display.waveplot(audio_signal, sr=sr)
    # iterate over list of text annotations and draw them
    if annotations is not None:
        for x,y,text in annotations:
        
            ax.annotate(
            text,
            xy=(x,y), xycoords='data',
            xytext=(10, -50), textcoords='offset pixels',
            horizontalalignment='right',
            color='black',
            fontsize=20,
            verticalalignment='bottom',
            arrowprops=dict(
                arrowstyle= '-|>',
                 color='black',
                 lw=1,
                 ls='-')
            ) 

In [ ]:
show_waveplot(audio_signal,sr_agent)

**HW Task (5 points)**: Populate word-level annotations for the plot function. Keep in mind that the y axis of the time-amplitude plot is different than the frequency axis of the Mel spectrum above.

In [ ]:
##
#
# YOUR CODE HERE
#
##

# create a full set of word-level transcriptions for the wave plot
utt_words = [(0.1, 0, 'This'),]

In [ ]:
show_waveplot(audio_signal,sr_agent,utt_words)

**HW Task (5 points)**: Describe what aspects of speech you can infer in this time series plot compared to the Mel spectrum above. Compare the two, when might looking at one vs the other help when working with audio data?

## Shifting to the Frequency Domain

In general, it is hard to directly work with waveforms in machine learning. As you can see, there are differences in magnitude, variable lengths, and patterns are hard to discern by the human eye. So, we often rely on signal processing tools to "standardize" waveforms. In speech, it is common to convert raw waveforms to the frequency domain with fourier transforms. We've been using a specialized transform similar to fourier transforms in our Mel spectrogram plots above. In this part of the homework we will build up our own implementation of a Mel spectrogram to understand the steps involved

### Short-time fourier spectrograms

If we think of an audio signal as a composition of sound waves of different frequencies, the fourier transform allows us to decompose a waveform into a set of basis frequencies with corresponding amplitudes. So we can equivalently represent a timeseries waveform in the frequency domain (i.e. a spectrum).

In [ ]:
# example of an fft on a segment of our audio signal
segment = audio_signal[:1000]
n_fft = 2048  # number of frequencies to decompose waveform into.
spectrum = np.abs(librosa.stft(segment, hop_length=len(segment)+1))

plt.figure(figsize=(8,5))
plt.plot(spectrum)
plt.title('Spectrum', fontsize=16)
plt.xlabel('Frequency', fontsize=14)
plt.ylabel('Amplitude', fontsize=14)
plt.show()

This would suffice if the frequencies of our audio signal were constant but as in our dataset, as speakers make utterances, signal frequencies change (i.e. they are non-periodic). If we can't faithfully summarize an audio signal with a single spectrum, we can use many time slices and look at the spectrum per-slice to understand how the signal changes over time. 

Imagine breaking up a waveform into overlapping windowed segments, computing the spectrum of each segment, and plot amplitudes over time. This is called a short-time fourier transform, abbreviated `stft`.

**HW Task (10 points):** We will program our own `stft` function. 

You cannot use the one in the `librosa` package. To compute fast fourier transforms, use `scipy.fft.fft`. The signature of your function should be: 

```
def simple_stft(y, hop_length=512, win_length=2048, n_fft=2048):
    """
    A simple short-time fourier transform. 
    
    Arguments
    ---------
    y: numpy array
       audio signal
    hop_length : integer [default: 512]
                 the number of samples to skip in between windows.
    win_length : integer [default: 2048]
                 the size of a single windowed segment.
    n_fft: integer [default: 2048]
           number of frequencies for fast fourier transforms.
    
    Outputs
    -------
    stft_matrix : numpy array
                  shape: n_windows x n_fft
                  contains amplitudes for the spectrogram.
    """
    # TODO
    stft_matrix = ...
    return stft_matrix
```

Concretely, given an audio signal of 
```
[ 1 2 3 4 5 6 7 8 9 10 ]
```
a `hop_length` of 2 and a `win_length` of 3 would result in the following windowed segments
```
[ 1 2 3 ] [ 3 4 5 ] [ 5 6 7 ] [ 7 8 9]
```
(here `n_windows = 3`). 

Each of these windows would then be put through a fast fourier transform and concatenated along the first dimension.

For comparison and debugging, here is the output of `librosa.stft`. Your solution will not match exactly since there are bells and whistles in their implementation that our simple one will not have. For example, due to a common issue known as aliasing, your output may have messier patterns that `librosa`'s will not have. However, the structure should look similar.

In the plot below, the x-axis is time (where index represents a windowed segment). The y-axis is frequency in hertz. Each cell represents amplitude.

In [ ]:
# NOTE: it is important to absolute value to remove complex values
lib_spec = np.abs(librosa.stft(audio_signal, hop_length=512, n_fft=2048, win_length=2048))

plt.figure(figsize=(20, 5))
plt.title('Librosa Spectrogram (Amplitude)', fontsize=20)
librosa.display.specshow(lib_spec, sr=sr_agent, x_axis='time', y_axis='log');
plt.colorbar(format='%+2.0f Amplitude');

# we often convert amplitude to decibels before visualizing because changes in are 
# much more discernable to the human eye
lib_spec_db = librosa.amplitude_to_db(lib_spec, ref=np.max)
plt.figure(figsize=(20, 5))
plt.title('Librosa Spectrogram (Decibels)', fontsize=20)
librosa.display.specshow(lib_spec_db, sr=sr_agent, x_axis='time', y_axis='log');
plt.colorbar(format='%+2.0f dB');

In [ ]:
##
#
# YOUR CODE HERE
#
##

# repurpose the processing/visualization code from the cell above to work with your stft function

### Using a Mel scale

In terms of the human auditory system, not all frequencies are perceived equally. In very low or high frequencies (in hertz), our ears are less capable at discriminating between different frequencies. In the spectrograms above, we have treated all frequencies equally. This makes them look like a warped version (along the frequency axis) of the Mel spectrograms above. 

The Mel-scale is a scale of pitches judged by human listeners to be equal in distance one from another. It is roughly linear between 0 and 1000hz and logarithmic above 1000hz, as human ears become less adept at differentiating frequencies. We can think of the mel scale as a weight to multiply each frequency in a normal spectrogram to up- and down-weight different components. We can visualize it below.

In [ ]:
mel_basis = librosa.filters.mel(sr_agent, n_fft=256, n_mels=128, fmax=fmax_dataset)

plt.figure(figsize=(10,5))
librosa.display.specshow(mel_basis, x_axis='linear',fmax=fmax_dataset)
plt.title('Mel filter bank', fontsize=20)
plt.colorbar()
plt.show()

## Mel frequency cepstral coefficients

We briefly discuss a second popular way to featurize timeseries audio signal. While less popular for modern speech processing e.g. neural networks, they are commonly used with more traditional statistical methods (e.g. Gaussian mixture models). We will revisit these in homework 3. 

A cepstral is a spectrum of a spectrum (e.g. nested fourier transform). MFCCs are a cepstral representation on the Mel scale. Usually, to compute MFCCs, you first do a fourier transform on the audio waveform, map to the Mel scale, followed by a second transform. 

The recursive transform has the effect of smoothing out pitch ripples (hence making signal look more "Gaussian", which is useful when training a Gausian mixture model). However, over-smoothing may remove important features relating to speaker identity or intonation that might be useful. With modern deep learning approaches, MFCCs are usually not the default choice of features. It's still good to try them when working on tasks with smaller datasets where invariance to speakers is helpful.

**HW Task (5 points):** Use `librosa` to compute and plot the MFCC for the `audio_signal` waveform. Use a `hop_length` of 512 and 20 cepstral coefficients. 

In [ ]:
##
#
# YOUR CODE AND PLOT HERE
#
##

A downside to MFCC is that it is less interpretable. It would hard to manually annotate this cepstrum with each utterance as we did for Mel Spectrograms.

### Pitch shifting audio signals

One thing we often do with speech data is shift pitches. This could be used as data augmentation technique (e.g. to prevent overfitting when training neural networks) or as standardization technique to obscure speaker identity (e.g. for privacy purposes).

In [ ]:
# recall our original snippet
ipd.Audio(audio_signal, rate=sr_agent)

We can use the `librosa.effects` library to shift our sample up and down. See the examples below and listen to the shifted samples.

In [ ]:
# shift up by four half steps
audio_u4hs = librosa.effects.pitch_shift(audio_signal, sr_agent, n_steps=4)
ipd.Audio(audio_u4hs, rate=sr_agent)

In [ ]:
# shift down by six half steps
audio_d6hs = librosa.effects.pitch_shift(audio_signal, sr_agent, n_steps=-6)
ipd.Audio(audio_d6hs, rate=sr_agent)

**HW Task (5 points):** Pitch shift the full agent and caller audio samples as best you can to make it sound like a single person is talking. Recombine both clips into a single waveform and use `ipd.Audio` to display your results. If you're successful, it should sound like just one speaker is performing both sides of the conversation. We may give some extra points for creative approaches! As a baseline, it's fine to simply try to make both speakers' mean F_0 approximately equal. Try to ensure you can still understand the utterances as you work to make the speakers sound similar.

In [ ]:
wav_agent, sr_agent = librosa.load('./sample/agent/0002f70f7386445b.wav')
wav_caller, sr_caller = librosa.load('./sample/caller/0002f70f7386445b.wav')

In [ ]:
##
#
# YOUR CODE HERE
#
##

wav_total = None # TODO: recombine both clips into a single waveform (the full warped conversation)

In [ ]:
ipd.Audio(wav_total, rate=sr_agent)

In [ ]:
plot_melspectrogram(wav_total, sr_agent)

---

## Questions?

Please post on Piazza or come to office hours!